In [1]:
import os
import numpy as np
import pandas as pd
import librosa as lr

from tqdm import tqdm_notebook as tn

In [2]:
tracks_df = pd.read_csv("./data/fma_metadata/tracks.csv", index_col=0)
tracks_df = tracks_df[["track_title", "genre_top"]]

In [3]:
tracks_df.head()

,track_title,genre_top
track_id,,
2,Food,Hip-Hop
3,Electric Ave,Hip-Hop
5,This World,Hip-Hop
10,Freeway,Pop
20,Spiritual Level,NaN


In [6]:
tracks_df.loc[255]

track_title    We Want to be Old
genre_top                   Rock
Name: 255, dtype: object

In [4]:
data_train = []
genres = {}

for track in tn(os.listdir("./data/fma_small/")):
    if track[-4:] == ".mp3":
        track_id = int(track[:-4])
        track_data = tracks_df.loc[track_id].values

        if track_data[1] not in genres:
            genres[track_data[1]] = len(genres)

        try:
            y, sr = lr.load(f"./data/fma_small/{track}", sr=None)
            y = y[:5*sr]

            hl = int((sr / 44100) * 2048)
            feature = lr.feature.melspectrogram(y, sr, n_mels=128, hop_length=hl)

            data_train.append([feature, genres[track_data[1]]])
        except Exception as e:
            pass
        
np.random.shuffle(data_train)

In [5]:
print("Track Data: ")
for i in range(5):
    print(f"X Shape: {str(data_train[i][0].shape):<15}Y: {data_train[i][1]}")

# print("Track Data: ")
# for i in range(10):
#     print(f"ID: {data_train[i][0]:<7}Shape: {str(data_train[i][1].shape):<15}Title: {data_train[i][2]:<25}Genre: {data_train[i][3]:<17}Genre ID: {data_train[i][4]}")

Track Data: 
X Shape: (128, 108)     Y: 0
X Shape: (128, 108)     Y: 5
X Shape: (128, 108)     Y: 2
X Shape: (128, 108)     Y: 3
X Shape: (128, 108)     Y: 5


In [6]:
print("Track Genres: ")
for k, v in genres.items():
    print(f"[{v}] {k}")

Track Genres: 
[0] Hip-Hop
[1] Pop
[2] Folk
[3] Experimental
[4] Rock
[5] International
[6] Electronic
[7] Instrumental


In [7]:
X = []
Y = []

for features, label in data_train:
    X.append(features)
    Y.append(label)
    
X = np.array(X).reshape(-1, 128, 108, 1)
Y = np.array(Y)

np.save("./data/X.npy", X)
np.save("./data/Y.npy", Y)